# GUI from managing DICOM file repairs

### Imports

In [1]:
from pathlib import Path
from functools import partial
from typing import Optional, Union, Any, Dict, Tuple, List, Set, NamedTuple

import PySimpleGUI as sg



In [ ]:
from dicom_repair_functions import scan_dicom_images, do_repairs


### Setup functions

#### Functions to build directory selectors

In [2]:
def file_selector(selection, file_k, frame_title,
                    starting_path=Path.cwd(), file_type=None):
    # Check all starting path possibilities
    if isinstance(starting_path, str):
        starting_path = Path(starting_path)
    if starting_path.exists():
        if starting_path.is_file():
            initial_file = starting_path.name
            initial_dir = starting_path.parent
        elif starting_path.is_dir():
            initial_dir = str(starting_path)
            initial_file = ''
    else:
        if not starting_path.parent.exists():
            starting_path = Path.cwd()
            initial_dir = str(starting_path)
            initial_file = ''
        else:
            initial_dir = starting_path.parent
            initial_file = ''

    # Set browser by selection type
    if 'read file' in selection:
        browse = sg.FileBrowse(initial_folder=initial_dir,
                                file_types=file_type)
        initial_text = initial_file
    elif 'save file' in selection:
        browse = sg.FileSaveAs(initial_folder=initial_dir,
                                file_types=file_type)
        initial_text = initial_file
    elif 'read files' in selection:
        browse = sg.FilesBrowse(initial_folder=initial_dir,
                                file_types=file_type)
        initial_text = initial_file
    elif 'dir' in selection:
        browse = sg.FolderBrowse(initial_folder=initial_dir)
        initial_text = initial_dir
    else:
        raise ValueError(f'{selection} is not a valid browser type')

    # Build the selector group
    file_selector_frame = sg.Frame(title=frame_title, layout=[
        [sg.InputText(key=file_k, default_text=initial_text, size=(90, 1)),
         browse]])

    return file_selector_frame


In [3]:
def make_file_selection_frame(starting_input_path: Path,
                              starting_output_path: Path)->sg.Frame:
    # Select Input Directory
    input_title = 'Select the directory containing DICOM files to be repaired.'
    input_path_def = dict(frame_title=input_title, file_k='input_folder',
                        selection='dir', starting_path=starting_input_path)
    input_path_selector = file_selector(**input_path_def)

    # Select Output Directory
    output_title = ''.join(['Select the directory where repaired DICOM files',
                            'will be saved.'])
    output_path_def = dict(frame_title=output_title, file_k='output_folder',
                        selection='dir',starting_path=starting_output_path)
    output_path_selector = file_selector(**output_path_def)

    # Should Sub-Directories be included?
    help_text = ''.join(['Iteratively search all subdirectories of the ',
                        'supplied directory for DICOM files'])
    include_sub_dir_check = sg.Checkbox('Include Sub-Directories?',
                                        tooltip=help_text, default=True,
                                        key='include_sub_dir',expand_x = True)
    # Widget layout
    path_selection_layout=[
        [input_path_selector],
        [include_sub_dir_check],
        [output_path_selector]]

    # Build the frame
    path_frame = sg.Frame(key='Dir Selection',
                        title='Directories',
                        title_location=sg.TITLE_LOCATION_TOP_LEFT,
                        layout=path_selection_layout)
    return path_frame


#### Function to make buttons

In [4]:
def make_actions_buttons()->sg.Column:
    help_text = 'Begin repairing DICOM files.'
    start_button = sg.Button(button_text='Repair', key='start_repair',
                             tooltip=help_text, button_color='blue',
                             disabled_button_color = None,
                             highlight_colors = None,
                             mouseover_colors = (None, None),
                             font = None,
                             size = (None, None), pad = None,
                             border_width = None, auto_size_button = None,
                             disabled = False, visible = True)
    actions_list = sg.Column([[start_button, sg.Quit(button_color='red')]])
    return actions_list


#### Functions to track progress
##### Build Status Frame

In [5]:
class ProgressBar(sg.ProgressBar):
    '''A progress bar that tracks it's own progress.
    '''
    def __init__(self, *args, value=0, **kwargs):
        '''Generate the progress bar and add the starting value.
        Args:
            value (int, optional): The starting value for the progress bar.
                Defaults to 0.
        '''
        super().__init__(*args, **kwargs)
        self.value = value

    def update(self, *args, **kwargs)->bool:
        '''Update the progress bar and set the current value, if it is given.

        Returns:
            bool: Returns True if update was OK. False means something wrong
                with window or it was closed.
        '''
        status = super().update(*args, **kwargs)
        # if current_count is given, update the value attribute.
        if 'current_count' in kwargs:
            current_count = kwargs['current_count']
        elif len(args) > 0:
            current_count = args[0]
        else:
            current_count = None
        if current_count is not None:
            self.value = current_count
        return status

    def increment(self, step_size=1):
        '''Increase the progress bar value by step_size.

        Args:
            step_size (int): the amount to increment the progress level by.

        Returns:
            bool: Returns True if update was OK. False means something wrong
                with window or it was closed.
        '''
        current_level = self.value
        if not current_level:
            current_level = 0
        new_level = current_level + step_size
        status = self.update(new_level)
        return status


In [6]:
def make_status_frame()->sg.Frame:
    '''Build the status reporting widget set.'''
    status_layout=[
        [sg.Multiline(key='Status', autoscroll=True,
                      size=(97,10), pad=(10,10))],
        [ProgressBar(max_value=100, orientation='h', size=(62, 30),
                        pad=(10,10), key='Progress')]
        ]
    status_frame = sg.Frame(title='Status',
                            title_location=sg.TITLE_LOCATION_TOP,
                            layout=status_layout)
    return status_frame


##### Status reporting function

In [7]:
def print_to_window(main_window: sg.Window, status_element: sg.Element, text: str):
    '''Status reporting function framework.

    Args:
        main_window (sg.Window): Window containing the widget.
        status_element (sg.Element): The widget element.
        text (str): _description_
    '''
    status_element.print(text, text_color=None, background_color=None)
    main_window.refresh()


#### Main window function

In [8]:
#%% Main Window
def make_window(starting_input_path: Path, starting_output_path: Path):
    '''This blocks out the main sections of the GUI.

    Returns:
        sg.Window: The main GUI window.
    '''
    path_frame = make_file_selection_frame(starting_input_path,
                                           starting_output_path)
    actions_group = make_actions_buttons()
    status_frame = make_status_frame()

    main_layout = [
        [path_frame],
        [actions_group],
        [status_frame]
        ]

    window = sg.Window('DICOM Repair', finalize=True, resizable=True,
                       layout=main_layout)
    return window


In [9]:
def get_file_paths(window):
    # Get file paths
    process_repairs = True
    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED:
            process_repairs = False
            window.close()
            break
        if event == 'Quit':
            process_repairs = False
            window.close()
            break
        if event == 'start_repair':
            process_repairs = True
            break
    values['process_repairs'] = process_repairs
    return values


In [11]:
def status_output(main_window: sg.Window, status_element: sg.Element,
                  progress_element: sg.Element, repair_log: List[str],
                  message='', value: int = None, max_count: int = None):
    '''Sends message to status widget and updates progress bar as needed.

    Args:
        main_window (sg.Window): Window containing the status and
            progress bar widgets.
        status_element (sg.Element): The status widget element object.
        progress_element (sg.Element): The progress bar widget element object.
        message (str, optional): The new repair log message. If the string is
            empty, status and repair_log will not be updated. Defaults to an
            empty string.
        repair_log (List[str]): A list of all repair log messages.  Used for
            stats analysis.
        value (int, optional): A specific progress level to be set. If supplied,
            the progress level will be modified to the given value regardless of
            previous values. Defaults to None.
        max_count (int, optional): The maximum (100%) progress level. If
            supplied, the maximum progress level will be set to this value and
            the progress level will be reset to 0. Defaults to None.
    '''
    if max_count:
        progress_element.update(0, max=max_count)
    if value:
        progress_element.update(value)
    if message:
        status_element.print(message, text_color=None, background_color=None)
        repair_log.append(message)
        if 'Checking file' in message:
            progress_element.increment()
    main_window.refresh()


# Testing

In [10]:
data_path = Path.cwd() / 'DICOM Test Data'
output_path = Path.cwd() / 'Output'

test_data_path = Path(r'.\DICOM Test Data\Invalid^Characters [Error3]\Series 012 [PT - MAC]')


### Build GUI

In [12]:
window = make_window(starting_input_path=test_data_path,
                     starting_output_path=output_path)
repair_log = []
status_update = partial(status_output, window, window['Status'],
                        window['Progress'], repair_log)
window.refresh()


### Select paths

In [13]:
values = get_file_paths(window)
if values['process_repairs']:
    data_path = Path(values['input_folder'])
    output_path = Path(values['output_folder'])
    include_subdirectories = values['include_sub_dir']


In [16]:
file_gen = scan_dicom_images(data_path, include_subdirectories)
number_of_files = len([f for f in file_gen])
number_of_files
status_update(f'Found {number_of_files} files', max_count=number_of_files)


In [17]:
status_update('Test message', value=50)


In [20]:
window['Progress'].increment()


True

In [21]:
window.close()
